### CS120 Mini-project on Recommendation Systems

#### Requirements:

Based upon your submissions of ratings over a wide range of movies, we selected the ratings from 22 students over 266 movies. We then constructed a rating matrix with dimension ($266 \times 22$), with its $(i,j)$th entry referring to the rating of user $j$ over movie $i$. Now $33$ entries with ratings in the matrix are missing; each of them is replaced with $-1$. 

Your assignment is to try different (at least one, but surely one might be inadequate) methods that we've introduced in the class, to recover (or you can imagine that those missing entries are what we try to figure out) those missing entries. 

- You need to illustrate the methods that you adopt in detail, including parameter settings and the reasoning of your choice. 
- Next, you need to write your code in the required box, and run them with output and write to files. 
- Besides, code annotation is also necessary. Too simple answer or annotation **receives no grade**.
- At last, you should compare different methods that you adopt (in terms of performance, for instance) and show your analysis. 

#### Dataset that you have:

In fact, more than the corrupted rating matrix, we also provide you with the names of movies (see *Movie-list* file in the foler *Basic-dataset*). The hint is that you could also think about making use of the correlation between different movies. For example, different movies may belong to similar genre(s) (or with the same actresses/actors). 

The above is just one possible way. At the same time, you may find another foler *Other-avaliable-dataset*. That is a much larger dataset with users' ratings over even more movies (from real-world, too). You can also make use of information there, e.g. see more people's rating over the same movie in our basic dataset; in other words, you can regard it as a potential / optional training set, although we don't require you to import it). 

Remember: we need the recovered matrix. Others are optional. 

#### Describe the method(s) here (at least one), which you adopt in your mini project to recover the missing pieces in the rating matrix. 

First, I just use the baseline model to get my hands dirty and get familiar with ipython notebook and numpy, as I use matlab in my previous homework. So the implementation of baseline predictor is not very hard, and we don't need involve any parameter. What we do is to minimize $\|Ab-c\|$, so I just calculate $A$ and $c$, and $b=A^{\dagger}c$ to finally get recover rating matrix. 

To take control of overfitting, I modified the baseline predictor with  regularization, which is $min \{\|Ab-c\| +  \|\lambda\|\}$. First, I just set lamda to 1, which generate better result compared with simple baseline model. However, to find the optimal lamda, I tried to use the cross validation techniques. Since I don't have enough time to fully understand the technique to select the optimal lamda, such as Leave-one-out computations, I just use ***sklearn*** module to obtain the lamda, which has already implemented cross validation to tunne the parameter.

Once I finished the baseline predictor, I try to use the neighborhood model to get more accurate prediction. And I use the movie-movie correlation.

---
Note: for convenience, I use the rating_matrix_to_rec.transpose() $\left(22 \times 266\right)$ in most part of my code, however, save the recover matrix in rating_matrix_to_rec form ($266 \times 22$).

#### Then for each method, fill up with your runnable code below.

In [9]:
#Method 1
import csv
import numpy as np

rating_matrix_to_rec = []

## Import the rating matrix to recover here
## as a [[..], [..], .., [..]] data structure.
with open('Basic-dataset/rating_matrix_to_rec.csv', 'r') as f:
    reader  = csv.reader(f)
    
    for row in reader:
        row = list(map(lambda val: float(val), row))
        rating_matrix_to_rec.append(row)
        
num_of_users = len(rating_matrix_to_rec[0])
num_of_movies= len(rating_matrix_to_rec)

rating_matrix = np.array(rating_matrix_to_rec)
matrix_rec    = np.zeros((num_of_users, num_of_movies)) 

rating_matrix_to_rec = []
## import the testing matrix to calculate RMSE
with open('Basic-dataset/ratings.csv', 'r') as f:
    reader  = csv.reader(f)
    
    for row in reader:
        row = list(map(lambda val: float(val), row))
        rating_matrix_to_rec.append(row)
        
test_rating_matrix = np.array(rating_matrix_to_rec).transpose()
        
## Now you may start here..

# count rated movies number and average rating
rating_number = (rating_matrix>0).sum()
rating_average = rating_matrix.sum()/rating_number

# build vector c
c = rating_matrix.ravel()[rating_matrix.ravel()>0] - rating_average

rating_matrix = rating_matrix.transpose()

# build matrix A
A = np.zeros((rating_number ,num_of_users+num_of_movies))

count = 0
for j in range(0, len(rating_matrix[0])):
    for i in range (0, len(rating_matrix)):
        if (rating_matrix[i][j]>0):
            A[count, i]=1
            A[count,num_of_users+j]=1
            count+=1

# calculate b
b = np.dot(np.linalg.pinv(A), c)

bu = b[0:num_of_users]
bi = b[num_of_users:].transpose();

bus = np.tile(bu.transpose(), (num_of_movies,1)).transpose()
bis = np.tile(bi, (num_of_users,1))

# use b's introduced bu's and bi's to recover matrix
matrix_baseline = bus+bis
matrix_baseline += rating_average
matrix_baseline[matrix_baseline>5] = 5
matrix_baseline[matrix_baseline<1] = 1

matrix_rec = matrix_baseline

## Ends up with the variable named 'matrix_rec' that you recover.

## results of the RMSE with correspondant results
print('Method:', 'baseline model with regularization')

print('Estimated training set:', matrix_rec[rating_matrix>0])

# calculate RMSE of training
RMSE_of_trn_set = np.sqrt(((matrix_rec[rating_matrix>0]-rating_matrix[rating_matrix>0])**2/rating_number).sum())
print('RMSE over training set:', RMSE_of_trn_set)

print('Estimated test set:', matrix_rec[rating_matrix==-1])

# calculate RMSE of testing data
RMSE_of_tst_set = np.sqrt(((matrix_rec[rating_matrix==-1]-test_rating_matrix[rating_matrix==-1])**2/(rating_matrix==-1).sum()).sum())
print('RMSE over test set:', RMSE_of_tst_set)

### Write output to file
your_name_in_English = 'xiegsh'
your_student_id = '97382505'

with open(your_name_in_English + '_' + your_student_id + '_rec_matrix_1.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerows(matrix_rec.transpose())

Method: baseline model with regularization
Estimated training set: [ 4.57596321  4.83545335  1.          1.          4.1800691   4.22231713
  3.90904872  4.          5.          4.10193859  5.          3.57145138
  2.34714067  3.60329957  4.15331828  3.94838543  4.37179886  3.          4.
  4.48732995  2.          2.39791359  3.94794489  4.12240267  3.67823673
  1.5         4.5         2.          4.87839051  1.66759739  3.2         4.8
  2.          3.4         2.96965764  1.97341152  2.81770747  1.8
  3.7680086   2.88038695  4.5         4.8         4.7         4.6         4.9
  4.08517303  4.7         4.7056548   1.          2.5         4.
  3.43753902  2.79608152  4.          4.72799514  3.11048277  2.          3.5
  3.6247567   4.23667415  3.70618694  3.          3.73803513  3.5
  4.28805384  3.8341948   1.5         1.92608147  3.          4.64783707
  4.          4.5         3.          4.5         3.5         3.5         4.
  4.5         2.          3.5         1.92608147  3.    

In [10]:
#Method 2
import csv
import numpy as np

rating_matrix_to_rec = []

## Import the rating matrix to recover here
## as a [[..], [..], .., [..]] data structure.
with open('Basic-dataset/rating_matrix_to_rec.csv', 'r') as f:
    reader  = csv.reader(f)
    
    for row in reader:
        row = list(map(lambda val: float(val), row))
        rating_matrix_to_rec.append(row)
        
num_of_users = len(rating_matrix_to_rec[0])
num_of_movies= len(rating_matrix_to_rec)

rating_matrix = np.array(rating_matrix_to_rec)
matrix_rec    = np.zeros((num_of_users, num_of_movies)) 

rating_matrix_to_rec = []
## import the testing matrix to calculate RMSE
with open('Basic-dataset/ratings.csv', 'r') as f:
    reader  = csv.reader(f)
    
    for row in reader:
        row = list(map(lambda val: float(val), row))
        rating_matrix_to_rec.append(row)
        
test_rating_matrix = np.array(rating_matrix_to_rec).transpose()
        
## Now you may start here..

# count rated movies number and average rating
rating_number = (rating_matrix>0).sum()
rating_average = rating_matrix.sum()/rating_number

# build vector c
c = rating_matrix.ravel()[rating_matrix.ravel()>0] - rating_average

rating_matrix = rating_matrix.transpose()

# build matrix A
A = np.zeros((rating_number ,num_of_users+num_of_movies))

count = 0
for j in range(0, len(rating_matrix[0])):
    for i in range (0, len(rating_matrix)):
        if (rating_matrix[i][j]>0):
            A[count, i]=1
            A[count,num_of_users+j]=1
            count+=1
            
# get optimal parameter lamda
# try to import sklearn module
try:
    from sklearn import linear_model
    reg = linear_model.RidgeCV(alphas=[1+0.0001*i for i in range(1,10000)])
    reg.fit(A,c)
    lamda = reg.alpha_
# else using pre-calculated value
except ImportError:
    lamda = 1.63018

# calculate b
b = np.dot(np.linalg.pinv( np.dot(A.transpose(), A) + lamda * np.eye( A.shape[1] ) ), np.dot(A.transpose(), c))

bu = b[0:num_of_users]
bi = b[num_of_users:].transpose();

bus = np.tile(bu.transpose(), (num_of_movies,1)).transpose()
bis = np.tile(bi, (num_of_users,1))

# use b's introduced bu's and bi's to recover matrix
matrix_baseline = bus+bis
matrix_baseline += rating_average
matrix_baseline[matrix_baseline>5] = 5
matrix_baseline[matrix_baseline<1] = 1

matrix_rec = matrix_baseline

## Ends up with the variable named 'matrix_rec' that you recover.

## results of the RMSE with correspondant results
print('Method:', 'baseline model with regularization')

print('Estimated training set:', matrix_rec[rating_matrix>0])

# calculate RMSE of training
RMSE_of_trn_set = np.sqrt(((matrix_rec[rating_matrix>0]-rating_matrix[rating_matrix>0])**2/rating_number).sum())
print('RMSE over training set:', RMSE_of_trn_set)

print('Estimated test set:', matrix_rec[rating_matrix==-1])

# calculate RMSE of testing data
RMSE_of_tst_set = np.sqrt(((matrix_rec[rating_matrix==-1]-test_rating_matrix[rating_matrix==-1])**2/(rating_matrix==-1).sum()).sum())
print('RMSE over test set:', RMSE_of_tst_set)

### Write output to file
your_name_in_English = 'xiegsh'
your_student_id = '97382505'

with open(your_name_in_English + '_' + your_student_id + '_rec_matrix_1.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerows(matrix_rec.transpose())

Method: baseline model with regularization
Estimated training set: [ 4.27933852  4.73210785  2.22269159  2.22269159  4.01322366  3.90035945
  3.77435798  3.72269159  4.22269159  4.05729641  4.22269159  3.64630379
  2.83122567  3.58269413  4.19746565  3.77713645  3.97995423  3.30712468
  3.80712468  3.95670963  2.80712468  2.35892316  3.66146081  3.79273638
  3.54237401  2.55712468  4.05712468  2.80712468  4.34947033  1.69745183
  3.4412469   4.2412469   2.8412469   3.5412469   3.22576859  2.4271676
  3.1319314   2.7412469   3.94288685  3.33491289  4.0912469   4.2412469
  4.1912469   4.1412469   4.2912469   3.97367409  4.1912469   4.50066317
  2.15784252  2.90784252  3.65784252  3.45234161  2.92072556  3.65784252
  4.3490002   3.2438097   2.65784252  3.40784252  3.5760781   3.92759827
  3.51660565  3.15784252  3.45299598  3.40784252  4.06776751  3.73786651
  2.4825161   2.16224056  3.2325161   4.01099168  3.7325161   3.9825161
  3.2325161   3.9825161   3.4825161   3.4825161   3.7325161 

In [9]:
#Method 3
import csv
import numpy as np

rating_matrix_to_rec = []

## Import the rating matrix to recover here
## as a [[..], [..], .., [..]] data structure.
with open('Basic-dataset/rating_matrix_to_rec.csv', 'r') as f:
    reader  = csv.reader(f)
    
    for row in reader:
        row = list(map(lambda val: float(val), row))
        rating_matrix_to_rec.append(row)
        
num_of_users = len(rating_matrix_to_rec[0])
num_of_movies= len(rating_matrix_to_rec)

rating_matrix = np.array(rating_matrix_to_rec)
matrix_rec    = np.zeros((num_of_users, num_of_movies)) 

rating_matrix_to_rec = []
## import the testing matrix to calculate RMSE
with open('Basic-dataset/ratings.csv', 'r') as f:
    reader  = csv.reader(f)
    
    for row in reader:
        row = list(map(lambda val: float(val), row))
        rating_matrix_to_rec.append(row)
        
test_rating_matrix = np.array(rating_matrix_to_rec).transpose()
        
## Now you may start here..

# count rated movies number and average rating
rating_number = (rating_matrix>0).sum()
rating_average = rating_matrix.sum()/rating_number

# build vector c
c = rating_matrix.ravel()[rating_matrix.ravel()>0] - rating_average

rating_matrix = rating_matrix.transpose()

# build matrix A
A = np.zeros((rating_number ,num_of_users+num_of_movies))

count = 0
for j in range(0, len(rating_matrix[0])):
    for i in range (0, len(rating_matrix)):
        if (rating_matrix[i][j]>0):
            A[count, i]=1
            A[count,num_of_users+j]=1
            count+=1

# calculate b
b = np.dot(np.linalg.pinv(A), c)

bu = b[0:num_of_users]
bi = b[num_of_users:].transpose();

bus = np.tile(bu.transpose(), (num_of_movies,1)).transpose()
bis = np.tile(bi, (num_of_users,1))

# use b's introduced bu's and bi's to recover matrix
matrix_baseline = bus+bis
matrix_baseline += rating_average
matrix_baseline[matrix_baseline>5] = 5
matrix_baseline[matrix_baseline<1] = 1

matrix_rec = matrix_baseline

# continue to build neighborhood modle

# build matrix_tilde
matrix_tilde = rating_matrix-matrix_baseline
matrix_tilde[rating_matrix<=0] = 0

# build matrix D
D = np.zeros((num_of_users,num_of_users))
for i in range(num_of_users):
    for j in range(num_of_users):
        index = np.logical_and((rating_matrix[i,:]>0), (rating_matrix[j,:]>0))
        ri = matrix_tilde[i, index]
        rj = matrix_tilde[j, index]
        if ri.size>1:
            D[i,j] =np.dot(ri,rj) / np.sqrt(np.dot(ri,ri) * np.dot(rj,rj))
        else:
            D[i,j]=np.nan
# wipe off self
D[np.eye(num_of_users)>0]=np.nan

# build neighborhood matrix
matrix_neighborhood = np.zeros((num_of_users,num_of_movies))
# select most 2 close neighbor
k=2
for j in range(num_of_users):
    index = np.argsort(np.abs(D[j,:]), axis=None)[0:k]
    for i in range(num_of_movies):
        index_tmp = index
        index_tmp = index_tmp[rating_matrix[index_tmp,i]>0]
        if index_tmp.size > 0:
            tmp = np.abs(D[index_tmp,j]).sum()
            if tmp*(tmp==tmp)>0:
                matrix_neighborhood[j,i] = np.dot(D[index_tmp,j], matrix_tilde[index_tmp,i])/tmp

matrix_rec = matrix_baseline+matrix_neighborhood
matrix_rec[matrix_rec>5] = 5
matrix_rec[matrix_rec<1] = 1

## Ends up with the variable named 'matrix_rec' that you recover.

## results of the RMSE with correspondant results
print('Method:', 'neighborhood model')

print('Estimated training set:', matrix_rec[rating_matrix>0])

# calculate RMSE of training
RMSE_of_trn_set = np.sqrt(((matrix_rec[rating_matrix>0]-rating_matrix[rating_matrix>0])**2/rating_number).sum())
print('RMSE over training set:', RMSE_of_trn_set)

print('Estimated test set:', matrix_rec[rating_matrix==-1])

# calculate RMSE of testing data
RMSE_of_tst_set = np.sqrt(((matrix_rec[rating_matrix==-1]-test_rating_matrix[rating_matrix==-1])**2/(rating_matrix==-1).sum()).sum())
print('RMSE over test set:', RMSE_of_tst_set)

### Write output to file
your_name_in_English = 'xiegsh'
your_student_id = '97382505'

with open(your_name_in_English + '_' + your_student_id + '_rec_matrix_1.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerows(matrix_rec.transpose())

Method: neighborhood model
Estimated training set: [ 4.57596321  4.83545335  1.          1.          4.1800691   4.22231713
  3.90904872  4.          5.          4.86525886  5.          3.3650862
  2.34714067  3.86526444  4.86526444  3.94838543  4.37179886  3.          4.
  5.          2.          2.39791359  3.39588977  5.          3.67823673
  1.5         4.5         2.          4.3297425   1.76910174  3.2         4.8
  2.          3.4         2.96965764  1.97341152  2.81770747  1.8
  3.7680086   3.1         4.5         4.8         4.7         4.6         4.9
  4.08517303  4.7         4.7056548   1.          2.5         4.
  3.43753902  2.79608152  4.          4.72799514  2.70344353  2.          3.5
  3.6247567   4.23667415  3.70618694  3.          3.39670043  3.5
  4.49673786  3.8341948   1.5         2.          3.          4.64783707
  4.          4.5         3.          4.5         3.5         3.5         4.
  4.5         2.          3.5         2.          3.          2.         

#### Comparison and your analysis:

##### Model 1
As we can see, the fitting accuracy of baseline model is very well for training data. which usually result in overfitting according to noise. So the RMSE over training set is highly optimal $0.454441117893$, and the RMSE over test set: $1.28871217194$. So we can introduce regulation to take control of the hindsight of this model, which will give us more general model. 

##### Model 2
When I added the penalty to the baseline model with simply $\lambda = 1$, the RMSE acted better on the testing set, which is about $1.23256350735$. Before applying the technique to determine optimal lamda, I just use the test data to directly find optimal lamda to compare with the cross validation result, see wether this technique is useful and efficient. The optimal lamda for test data is about $\lambda = 1.428848$ , which result in RMSE over test set $1.23078170678$. Myself program obtain lamda is about $\lambda = 1.63018$, which result RMSE over test set is $1.231041284233741$, which still reduced the error hugely.

##### Model 3
The RMSE over test set of neighborhood model is about $1.28697937343$, which is a little impovement. In my opnion, the data is sparse and with much noise.


**Due date**: 23:59, Nov. 18, 2016. 
Overdue submission receives no grade.